In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data preprocessing

In [2]:
import requests
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filename = "tinyshakespeare.txt"

r = requests.get(url)
open(filename, "w", encoding="utf-8").write(r.text)

text = open(filename, "r", encoding="utf-8").read()

print(f"Length of dataset: {len(text):,} characters")
print(text[:500]) 


Length of dataset: 1,115,394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

2025-10-20 12:00:21.773081: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760961621.944232      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760961622.000043      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
tokenizer=Tokenizer()

In [5]:
tokenizer.fit_on_texts([text])

In [6]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'to': 3,
 'i': 4,
 'of': 5,
 'you': 6,
 'my': 7,
 'a': 8,
 'that': 9,
 'in': 10,
 'is': 11,
 'not': 12,
 'for': 13,
 'with': 14,
 'me': 15,
 'it': 16,
 'be': 17,
 'your': 18,
 'his': 19,
 'but': 20,
 'this': 21,
 'he': 22,
 'have': 23,
 'as': 24,
 'thou': 25,
 'him': 26,
 'so': 27,
 'what': 28,
 'thy': 29,
 'will': 30,
 'by': 31,
 'no': 32,
 'all': 33,
 'king': 34,
 'we': 35,
 'shall': 36,
 'her': 37,
 'if': 38,
 'our': 39,
 'are': 40,
 'do': 41,
 'thee': 42,
 'lord': 43,
 'now': 44,
 'on': 45,
 'good': 46,
 'from': 47,
 'come': 48,
 'sir': 49,
 'or': 50,
 'which': 51,
 'more': 52,
 'then': 53,
 'at': 54,
 'o': 55,
 'would': 56,
 'was': 57,
 'they': 58,
 'how': 59,
 'well': 60,
 'here': 61,
 'she': 62,
 'than': 63,
 'their': 64,
 'them': 65,
 'duke': 66,
 'am': 67,
 'hath': 68,
 'say': 69,
 'let': 70,
 'when': 71,
 'one': 72,
 "i'll": 73,
 'go': 74,
 'love': 75,
 'were': 76,
 'may': 77,
 'us': 78,
 'make': 79,
 'like': 80,
 'upon': 81,
 'yet': 82,
 'richard': 83,

In [7]:
last_index=len(tokenizer.word_index)
last_index

12632

In [8]:
input_sequences=[]

In [9]:
for sentence in text.split("\n"):
    tokenized_sentence=tokenizer.texts_to_sequences([sentence])[0]
    for i in range (1,len(tokenized_sentence)):
        n_gram=tokenized_sentence[:i+1]
        input_sequences.append(n_gram)

In [10]:
input_sequences[:5]

[[88, 269],
 [139, 35],
 [139, 35, 969],
 [139, 35, 969, 143],
 [139, 35, 969, 143, 668]]

In [11]:
max_len=max([len(x) for x in input_sequences])
max_len

16

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
padded_input_sequences= pad_sequences(input_sequences,maxlen=max_len,padding='pre')

In [14]:
padded_input_sequences[:5]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,  88, 269],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0, 139,  35],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        139,  35, 969],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 139,
         35, 969, 143],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 139,  35,
        969, 143, 668]], dtype=int32)

In [15]:
X=padded_input_sequences[:,:-1]
X

array([[   0,    0,    0, ...,    0,    0,   88],
       [   0,    0,    0, ...,    0,    0,  139],
       [   0,    0,    0, ...,    0,  139,   35],
       ...,
       [   0,    0,    0, ...,    0,    0, 1062],
       [   0,    0,    0, ...,    0, 1062,   25],
       [   0,    0,    0, ..., 1062,   25,  131]], dtype=int32)

In [16]:
y=padded_input_sequences[:,-1]
y

array([ 269,   35,  969, ...,   25,  131, 1828], dtype=int32)

In [17]:
from tensorflow.keras.utils import to_categorical
to_categorical(y,num_classes=len(tokenizer.word_index)+1)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# model architecture and training

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import numpy as np

In [19]:
model = Sequential([
    Embedding(last_index+1, 256, input_shape=(max_len,)),
    LSTM(256, dropout=0.3),
    Dense(last_index+1, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1760961635.243362      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [20]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.005, clipnorm=1.0)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 16, 256)        │     3,234,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 12633)          │     3,246,681 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,006,041 (26.73 MB)

 Trainable params: 7,006,041 (26.73 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_loss',
    save_best_only=True
)

In [23]:
history = model.fit(
    X, y,
    validation_split=0.1,
    epochs=100,
    batch_size=64,
    callbacks=[early_stop, checkpoint]
)

Epoch 1/100


I0000 00:00:1760961640.591434      59 cuda_dnn.cc:529] Loaded cuDNN version 90300


2410/2410 ━━━━━━━━━━━━━━━━━━━━ 35s 13ms/step - accuracy: 0.0606 - loss: 6.7796 - val_accuracy: 0.0874 - val_loss: 6.4136
Epoch 2/100
2410/2410 ━━━━━━━━━━━━━━━━━━━━ 30s 12ms/step - accuracy: 0.1074 - loss: 5.6905 - val_accuracy: 0.0945 - val_loss: 6.5035
Epoch 3/100
2410/2410 ━━━━━━━━━━━━━━━━━━━━ 30s 12ms/step - accuracy: 0.1275 - loss: 5.1614 - val_accuracy: 0.0963 - val_loss: 6.7857
Epoch 4/100
2410/2410 ━━━━━━━━━━━━━━━━━━━━ 30s 12ms/step - accuracy: 0.1489 - loss: 4.7524 - val_accuracy: 0.0880 - val_loss: 7.0818
Epoch 5/100
2410/2410 ━━━━━━━━━━━━━━━━━━━━ 30s 12ms/step - accuracy: 0.1676 - loss: 4.4338 - val_accuracy: 0.0831 - val_loss: 7.4005
Epoch 6/100
2410/2410 ━━━━━━━━━━━━━━━━━━━━ 30s 12ms/step - accuracy: 0.1851 - loss: 4.2281 - val_accuracy: 0.0842 - val_loss: 7.6875
Epoch 7/100
2410/2410 ━━━━━━━━━━━━━━━━━━━━ 30s 12ms/step - accuracy: 0.1968 - loss: 4.0906 - val_accuracy: 0.0782 - val_loss: 7.9318
Epoch 8/100
2410/2410 ━━━━━━━━━━━━━━━━━━━━ 30s 12ms/step - accuracy: 0.2102 - los

# prediction

In [24]:
text="royal"
# tokenize
tokenized_text=tokenizer.texts_to_sequences([text])[0]
# padding
padded_text=pad_sequences([tokenized_text],maxlen=max_len,padding='pre')

In [25]:
# predict
position=np.argmax(model.predict(padded_text))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step


In [26]:
for word,index in tokenizer.word_index.items():
    if index==position:
        print(word)

good


In [27]:
num_words=10

In [28]:
# prediction multiple words
text="speech"
for i in range(num_words):
    # tokenize
    tokenized_text=tokenizer.texts_to_sequences([text])[0]
    # padding
    padded_text=pad_sequences([tokenized_text],maxlen=max_len,padding='pre')
    # predict
    position=np.argmax(model.predict(padded_text))
    for word,index in tokenizer.word_index.items():
        if index==position:
            text=text+" "+word
            print(word)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
of
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
king
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
of
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
king
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
my
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
lord
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
i


In [29]:
text

'speech of the king of the king and my lord i'

now we can see the issue with n gram also